# Step 0: Download the Data

In [2]:
# Download file
!wget https://fss.pass3exceed4.com/FIN7790GroupProject_SHh5bb.zip

# Unzip the download file
import zipfile

f = zipfile.ZipFile("./FIN7790GroupProject_SHh5bb.zip",'r')
for file in f.namelist():
    f.extract(file,"./")
f.close()

--2024-03-03 08:47:44--  https://fss.pass3exceed4.com/FIN7790GroupProject_SHh5bb.zip
Resolving fss.pass3exceed4.com (fss.pass3exceed4.com)... 43.134.239.248
Connecting to fss.pass3exceed4.com (fss.pass3exceed4.com)|43.134.239.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1416350 (1.4M) [application/zip]
Saving to: ‘FIN7790GroupProject_SHh5bb.zip’

FIN7790GroupProject 100%[===================>]   1.35M  1.31MB/s    in 1.0s    

2024-03-03 08:47:46 (1.31 MB/s) - ‘FIN7790GroupProject_SHh5bb.zip’ saved [1416350/1416350]



# Step 1: Data Prepare - Label Encoding

In [3]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import pandas as pd
import json
from tqdm import tqdm

# Load Data
df = pd.read_csv(r"/content/FIN7790 Group Project/train.csv")

le_dict = {}

# Encoded categorical column
df_encoded = df.copy()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
for col in tqdm(categorical_cols, desc='Label Encoding....'):
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    le_dict[col] = le

print("Label Encoding Finished.")

# Save the encoded dataset
df_encoded.to_csv('Encoded_train_data.csv', index=False)

# Gather encoding mapping information
encoded_mappings = {}
for col in tqdm(categorical_cols, desc='Creating Mapping Json...'):
    le = le_dict[col]
    encoded_mappings[col] = list(le.classes_)

# Save encoding mapping information as a JSON file
with open('encoded_mappings.json', 'w') as f:
    json.dump(encoded_mappings, f)

print("\nMapping JSON file created finished.")

Label Encoding....: 100%|██████████| 4/4 [00:00<00:00, 270.83it/s]


Label Encoding Finished.


Creating Mapping Json...: 100%|██████████| 4/4 [00:00<00:00, 16627.57it/s]


Mapping JSON file created finished.


# Step 2: Correlation Calculation for All Value

In [8]:
import pandas as pd

train = pd.read_csv(r"Encoded_train_data.csv")

float_columns = train.select_dtypes(include=['float', 'int']).columns
correlation_matrix = train[float_columns].corr()

for col in train[float_columns].columns:
  target_column = col
  target_correlation = correlation_matrix[target_column]
  target_correlation = target_correlation.drop(target_column, axis=0)
  highest_corr_column = target_correlation.idxmax()
  highest_corr_value = target_correlation.max()
  if highest_corr_value > 0.0001:
    print(f"The column with the highest correlation to '{target_column}' is '{highest_corr_column}' with a correlation coefficient of {highest_corr_value}.")

The column with the highest correlation to 'Id' is 'Q10_OPERATING_INCOME' with a correlation coefficient of 0.07601448321909797.
The column with the highest correlation to 'industry' is 'sector' with a correlation coefficient of 0.21633335047243568.
The column with the highest correlation to 'sector' is 'industry' with a correlation coefficient of 0.21633335047243568.
The column with the highest correlation to 'fullTimeEmployees' is 'totalRevenue' with a correlation coefficient of 0.6470607026312295.
The column with the highest correlation to 'auditRisk' is 'overallRisk' with a correlation coefficient of 0.5276372666910631.
The column with the highest correlation to 'boardRisk' is 'overallRisk' with a correlation coefficient of 0.779076916487778.
The column with the highest correlation to 'compensationRisk' is 'overallRisk' with a correlation coefficient of 0.7730192502607067.
The column with the highest correlation to 'shareHolderRightsRisk' is 'overallRisk' with a correlation coeffic

# Sub Step 2: Correlation After Drop the Current Drop List

In [ ]:
import pandas as pd

train = pd.read_csv(r"Encoded_train_data.csv")

train = train.drop(columns_to_drop, axis=1)

float_columns = train.select_dtypes(include=['float', 'int']).columns
correlation_matrix = train[float_columns].corr()

for col in train[float_columns].columns:
  target_column = col
  target_correlation = correlation_matrix[target_column]
  target_correlation = target_correlation.drop(target_column, axis=0)
  highest_corr_column = target_correlation.idxmax()
  highest_corr_value = target_correlation.max()
  if highest_corr_value > 0.0001:
    print(f"The column with the highest correlation to '{target_column}' is '{highest_corr_column}' with a correlation coefficient of {highest_corr_value}.")

# Step 3.1: Baseline calculate

In this part, we using <keyborad> HistGradientBoostingRegressor </keyboard> to calculate the R2 for the 9 target element to build the base line.


In [9]:
import pandas as pd
import numpy as np

In [10]:
df = train = pd.read_csv(r"Encoded_train_data.csv")

In [11]:
target_columns = ['Q0_TOTAL_ASSETS',
 'Q0_TOTAL_LIABILITIES',
 'Q0_TOTAL_STOCKHOLDERS_EQUITY',
 'Q0_GROSS_PROFIT',
 'Q0_COST_OF_REVENUES',
 'Q0_REVENUES',
 'Q0_OPERATING_INCOME',
 'Q0_OPERATING_EXPENSES',
 'Q0_EBITDA']

In [12]:
columns_to_drop = []

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings("ignore")

result_1 = []

for i in target_columns:
  print(f"Current Target is {i}")

  X = train.drop(list(target_columns), axis=1)
  X = pd.get_dummies(X)
  y = train[i]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  model = HistGradientBoostingRegressor(max_iter=100, learning_rate=0.1, random_state=42)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)

  # 计算均方误差 (MSE)
  mse = mean_squared_error(y_test, y_pred)
  print(f"Mean Squared Error (MSE): {mse}")

  # 计算均方根误差 (RMSE)
  rmse = np.sqrt(mse)
  print(f"Root Mean Squared Error (RMSE): {rmse}")

  # 计算 R² 分数
  r2 = r2_score(y_test, y_pred)
  print(f"R² Score: {r2}")
  print("----------------------------------------")
  result_1.append({
      "Target Element": i,
      "R2": r2
  })

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


Current Target is Q0_TOTAL_ASSETS
Mean Squared Error (MSE): 2.9286895043241947e+19
Root Mean Squared Error (RMSE): 5411736786.212163
R² Score: 0.9277629396435301
----------------------------------------
Current Target is Q0_TOTAL_LIABILITIES
Mean Squared Error (MSE): 1.1493490431581387e+19
Root Mean Squared Error (RMSE): 3390205072.2015896
R² Score: 0.9102645365949589
----------------------------------------
Current Target is Q0_TOTAL_STOCKHOLDERS_EQUITY
Mean Squared Error (MSE): 1.4226575421625235e+19
Root Mean Squared Error (RMSE): 3771813280.3235683
R² Score: 0.8998033323590439
----------------------------------------
Current Target is Q0_GROSS_PROFIT
Mean Squared Error (MSE): 1.128144946051984e+17
Root Mean Squared Error (RMSE): 335878690.31124675
R² Score: 0.9401215301649868
----------------------------------------
Current Target is Q0_COST_OF_REVENUES
Mean Squared Error (MSE): 2.852882675284163e+18
Root Mean Squared Error (RMSE): 1689047860.566468
R² Score: 0.7524182411374529
---

# Step 3.2: Drop useless parament - [Id]

In this part, we will drop the useless parament Id, since Id don't explain any other value's, and also should not be a input value of model, so it can be safe to drop.


In [14]:
import pandas as pd
import numpy as np

In [15]:
df = train = pd.read_csv(r"Encoded_train_data.csv")

In [16]:
target_columns = ['Q0_TOTAL_ASSETS',
 'Q0_TOTAL_LIABILITIES',
 'Q0_TOTAL_STOCKHOLDERS_EQUITY',
 'Q0_GROSS_PROFIT',
 'Q0_COST_OF_REVENUES',
 'Q0_REVENUES',
 'Q0_OPERATING_INCOME',
 'Q0_OPERATING_EXPENSES',
 'Q0_EBITDA']

In [17]:
columns_to_drop = ['Id']

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings("ignore")

result_2 = []

for i in target_columns:
  print(f"Current Target is {i}")

  X = train.drop(list(target_columns) + list(columns_to_drop), axis=1)
  X = pd.get_dummies(X)
  y = train[i]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  model = HistGradientBoostingRegressor(max_iter=100, learning_rate=0.1, random_state=42)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)

  # 计算均方误差 (MSE)
  mse = mean_squared_error(y_test, y_pred)
  print(f"Mean Squared Error (MSE): {mse}")

  # 计算均方根误差 (RMSE)
  rmse = np.sqrt(mse)
  print(f"Root Mean Squared Error (RMSE): {rmse}")

  # 计算 R² 分数
  r2 = r2_score(y_test, y_pred)
  print(f"R² Score: {r2}")
  print("----------------------------------------")
  result_2.append({
      "Target Element": i,
      "R2": r2
  })

Current Target is Q0_TOTAL_ASSETS
Mean Squared Error (MSE): 2.9972897258393637e+19
Root Mean Squared Error (RMSE): 5474750885.510101
R² Score: 0.9260708933085594
----------------------------------------
Current Target is Q0_TOTAL_LIABILITIES
Mean Squared Error (MSE): 1.1407783727867195e+19
Root Mean Squared Error (RMSE): 3377541077.15468
R² Score: 0.9109336919590751
----------------------------------------
Current Target is Q0_TOTAL_STOCKHOLDERS_EQUITY
Mean Squared Error (MSE): 1.37856839399613e+19
Root Mean Squared Error (RMSE): 3712907747.3001266
R² Score: 0.9029084968800051
----------------------------------------
Current Target is Q0_GROSS_PROFIT
Mean Squared Error (MSE): 1.0991929306275336e+17
Root Mean Squared Error (RMSE): 331540786.42416435
R² Score: 0.9416582142482889
----------------------------------------
Current Target is Q0_COST_OF_REVENUES
Mean Squared Error (MSE): 2.841127027755273e+18
Root Mean Squared Error (RMSE): 1685564305.4346142
R² Score: 0.7534384316685903
-----

# Step 3.3: Drop high correlation columns - [TOTAL_LIABILITIES_AND_EQUITY, fiscal_year_end, TOTAL_CURRENT_ASSETS, TOTAL_NONCURRENT_ASSETS, TOTAL_CURRENT_LIABILITIES, TOTAL_NONCURRENT_LIABILITIES]

In this part, we will drop the useless parament Id, since Id don't explain any other value's, and also should not be a input value of model, so it can be safe to drop.


In [1]:
import pandas as pd
import numpy as np

In [4]:
df = train = pd.read_csv(r"Encoded_train_data.csv")

In [5]:
target_columns = ['Q0_TOTAL_ASSETS',
 'Q0_TOTAL_LIABILITIES',
 'Q0_TOTAL_STOCKHOLDERS_EQUITY',
 'Q0_GROSS_PROFIT',
 'Q0_COST_OF_REVENUES',
 'Q0_REVENUES',
 'Q0_OPERATING_INCOME',
 'Q0_OPERATING_EXPENSES',
 'Q0_EBITDA']

In [6]:
columns_to_drop = ['Id']
for i in range(1,11):
  columns_to_drop.append(f"Q{i}_TOTAL_LIABILITIES_AND_EQUITY")
  columns_to_drop.append(f"Q{i}_fiscal_year_end")
  columns_to_drop.append(f"Q{i}_TOTAL_CURRENT_ASSETS")
  columns_to_drop.append(f"Q{i}_TOTAL_NONCURRENT_ASSETS")
  columns_to_drop.append(f"Q{i}_TOTAL_CURRENT_LIABILITIES")
  columns_to_drop.append(f"Q{i}_TOTAL_NONCURRENT_LIABILITIES")

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings("ignore")

result_3 = []

for i in target_columns:
  print(f"Current Target is {i}")

  X = train.drop(list(target_columns) + list(columns_to_drop), axis=1)
  X = pd.get_dummies(X)
  y = train[i]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  model = HistGradientBoostingRegressor(max_iter=100, learning_rate=0.1, random_state=42)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)

  # 计算均方误差 (MSE)
  mse = mean_squared_error(y_test, y_pred)
  print(f"Mean Squared Error (MSE): {mse}")

  # 计算均方根误差 (RMSE)
  rmse = np.sqrt(mse)
  print(f"Root Mean Squared Error (RMSE): {rmse}")

  # 计算 R² 分数
  r2 = r2_score(y_test, y_pred)
  print(f"R² Score: {r2}")
  print("----------------------------------------")
  result_3.append({
      "Target Element": i,
      "R2": r2
  })

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


Current Target is Q0_TOTAL_ASSETS
Mean Squared Error (MSE): 3.3414206035680068e+19
Root Mean Squared Error (RMSE): 5780502230.40179
R² Score: 0.9175827954926917
----------------------------------------
Current Target is Q0_TOTAL_LIABILITIES
Mean Squared Error (MSE): 8.107376795378104e+18
Root Mean Squared Error (RMSE): 2847345570.0666375
R² Score: 0.9367016296691315
----------------------------------------
Current Target is Q0_TOTAL_STOCKHOLDERS_EQUITY
Mean Squared Error (MSE): 9.78616662751453e+18
Root Mean Squared Error (RMSE): 3128284933.875834
R² Score: 0.9310767872101103
----------------------------------------
Current Target is Q0_GROSS_PROFIT
Mean Squared Error (MSE): 1.4501275520524176e+17
Root Mean Squared Error (RMSE): 380805403.32989204
R² Score: 0.9230316820667731
----------------------------------------
Current Target is Q0_COST_OF_REVENUES
Mean Squared Error (MSE): 2.936286040155602e+18
Root Mean Squared Error (RMSE): 1713559465.018825
R² Score: 0.7451802457060884
-------

# Step 3.4: Drop high correlation columns - [TOTAL_LIABILITIES_AND_EQUITY, fiscal_year_end, TOTAL_CURRENT_ASSETS, TOTAL_NONCURRENT_ASSETS, TOTAL_CURRENT_LIABILITIES, TOTAL_NONCURRENT_LIABILITIES, TargetHighPrice, TargetLowPrice, TargetMediaPrice]

In this part, we will drop the useless parament Id, since Id don't explain any other value's, and also should not be a input value of model, so it can be safe to drop.

In [8]:
import pandas as pd
import numpy as np

In [9]:
df = train = pd.read_csv(r"Encoded_train_data.csv")

In [10]:
target_columns = ['Q0_TOTAL_ASSETS',
 'Q0_TOTAL_LIABILITIES',
 'Q0_TOTAL_STOCKHOLDERS_EQUITY',
 'Q0_GROSS_PROFIT',
 'Q0_COST_OF_REVENUES',
 'Q0_REVENUES',
 'Q0_OPERATING_INCOME',
 'Q0_OPERATING_EXPENSES',
 'Q0_EBITDA']

In [11]:
columns_to_drop = ['Id', 'targetHighPrice', 'targetLowPrice', 'targetMedianPrice']
for i in range(1,11):
  columns_to_drop.append(f"Q{i}_TOTAL_LIABILITIES_AND_EQUITY")
  columns_to_drop.append(f"Q{i}_fiscal_year_end")
  columns_to_drop.append(f"Q{i}_TOTAL_CURRENT_ASSETS")
  columns_to_drop.append(f"Q{i}_TOTAL_NONCURRENT_ASSETS")
  columns_to_drop.append(f"Q{i}_TOTAL_CURRENT_LIABILITIES")
  columns_to_drop.append(f"Q{i}_TOTAL_NONCURRENT_LIABILITIES")

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings("ignore")

result_4 = []

for i in target_columns:
  print(f"Current Target is {i}")

  X = train.drop(list(target_columns) + list(columns_to_drop), axis=1)
  X = pd.get_dummies(X)
  y = train[i]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  model = HistGradientBoostingRegressor(max_iter=100, learning_rate=0.1, random_state=42)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)

  # 计算均方误差 (MSE)
  mse = mean_squared_error(y_test, y_pred)
  print(f"Mean Squared Error (MSE): {mse}")

  # 计算均方根误差 (RMSE)
  rmse = np.sqrt(mse)
  print(f"Root Mean Squared Error (RMSE): {rmse}")

  # 计算 R² 分数
  r2 = r2_score(y_test, y_pred)
  print(f"R² Score: {r2}")
  print("----------------------------------------")
  result_4.append({
      "Target Element": i,
      "R2": r2
  })

Current Target is Q0_TOTAL_ASSETS
Mean Squared Error (MSE): 3.30849968644068e+19
Root Mean Squared Error (RMSE): 5751955916.417197
R² Score: 0.9183948004095687
----------------------------------------
Current Target is Q0_TOTAL_LIABILITIES
Mean Squared Error (MSE): 8.288187381287685e+18
Root Mean Squared Error (RMSE): 2878921218.3190575
R² Score: 0.9352899504397696
----------------------------------------
Current Target is Q0_TOTAL_STOCKHOLDERS_EQUITY
Mean Squared Error (MSE): 1.0283800143260086e+19
Root Mean Squared Error (RMSE): 3206836469.6785035
R² Score: 0.92757199294259
----------------------------------------
Current Target is Q0_GROSS_PROFIT
Mean Squared Error (MSE): 1.2863412261456142e+17
Root Mean Squared Error (RMSE): 358655994.8119666
R² Score: 0.9317249573498113
----------------------------------------
Current Target is Q0_COST_OF_REVENUES
Mean Squared Error (MSE): 2.967033143587125e+18
Root Mean Squared Error (RMSE): 1722507806.538805
R² Score: 0.7425119193800691
--------

# Step 3.5: Drop high correlation columns - [TOTAL_LIABILITIES_AND_EQUITY, fiscal_year_end, TOTAL_CURRENT_ASSETS, TOTAL_NONCURRENT_ASSETS, TOTAL_CURRENT_LIABILITIES, TOTAL_NONCURRENT_LIABILITIES, TargetHighPrice, TargetLowPrice, TargetMediaPrice]

In this part, we will drop the useless parament Id, since Id don't explain any other value's, and also should not be a input value of model, so it can be safe to drop.

In [13]:
import pandas as pd
import numpy as np

In [14]:
df = train = pd.read_csv(r"Encoded_train_data.csv")

In [15]:
target_columns = ['Q0_TOTAL_ASSETS',
 'Q0_TOTAL_LIABILITIES',
 'Q0_TOTAL_STOCKHOLDERS_EQUITY',
 'Q0_GROSS_PROFIT',
 'Q0_COST_OF_REVENUES',
 'Q0_REVENUES',
 'Q0_OPERATING_INCOME',
 'Q0_OPERATING_EXPENSES',
 'Q0_EBITDA']

In [16]:
columns_to_drop = ['Id', 'targetHighPrice', 'targetLowPrice', 'targetMedianPrice', 'Q9_TOTAL_ASSETS', 'Q10_TOTAL_LIABILITIES']
for i in range(1,11):
  columns_to_drop.append(f"Q{i}_TOTAL_LIABILITIES_AND_EQUITY")
  columns_to_drop.append(f"Q{i}_fiscal_year_end")
  columns_to_drop.append(f"Q{i}_TOTAL_CURRENT_ASSETS")
  columns_to_drop.append(f"Q{i}_TOTAL_NONCURRENT_ASSETS")
  columns_to_drop.append(f"Q{i}_TOTAL_CURRENT_LIABILITIES")
  columns_to_drop.append(f"Q{i}_TOTAL_NONCURRENT_LIABILITIES")

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings("ignore")

result_5 = []

for i in target_columns:
  print(f"Current Target is {i}")

  X = train.drop(list(target_columns) + list(columns_to_drop), axis=1)
  X = pd.get_dummies(X)
  y = train[i]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  model = HistGradientBoostingRegressor(max_iter=100, learning_rate=0.1, random_state=42)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)

  # 计算均方误差 (MSE)
  mse = mean_squared_error(y_test, y_pred)
  print(f"Mean Squared Error (MSE): {mse}")

  # 计算均方根误差 (RMSE)
  rmse = np.sqrt(mse)
  print(f"Root Mean Squared Error (RMSE): {rmse}")

  # 计算 R² 分数
  r2 = r2_score(y_test, y_pred)
  print(f"R² Score: {r2}")
  print("----------------------------------------")
  result_5.append({
      "Target Element": i,
      "R2": r2
  })

Current Target is Q0_TOTAL_ASSETS
Mean Squared Error (MSE): 3.1348282225355366e+19
Root Mean Squared Error (RMSE): 5598953672.37088
R² Score: 0.9226784624371714
----------------------------------------
Current Target is Q0_TOTAL_LIABILITIES
Mean Squared Error (MSE): 8.890649506292716e+18
Root Mean Squared Error (RMSE): 2981719219.895246
R² Score: 0.9305862254666525
----------------------------------------
Current Target is Q0_TOTAL_STOCKHOLDERS_EQUITY
Mean Squared Error (MSE): 1.019601374618253e+19
Root Mean Squared Error (RMSE): 3193119751.306319
R² Score: 0.9281902657307135
----------------------------------------
Current Target is Q0_GROSS_PROFIT
Mean Squared Error (MSE): 1.4336832578912926e+17
Root Mean Squared Error (RMSE): 378640100.608915
R² Score: 0.9239044947096262
----------------------------------------
Current Target is Q0_COST_OF_REVENUES
Mean Squared Error (MSE): 2.9390998136309043e+18
Root Mean Squared Error (RMSE): 1714380300.175811
R² Score: 0.744936057961499
---------